# Exercises

## Acquire (`acquire.py`)

### Zillow
**For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.**

**You will have a** 
`zillow.ipynb` 
**file and a helper file for each section in the pipeline.**

In [1]:
###############################################################################
### python imports                                                          ###
###############################################################################

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import LassoCV

import warnings
warnings.filterwarnings("ignore")

# from math import sqrt
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
###############################################################################
### local imports                                                           ###
###############################################################################

import acquire as acq
import prepare as prep

from debug import local_settings, timeifdebug, timeargsifdebug, frame_splain
from dfo import DFO, set_dfo

In [3]:
local_settings.splain=True
local_settings.debug=True
splain = local_settings.splain
debug = local_settings.debug

In [4]:
from wrangle_zillow import get_zillow_data, prep_zillow_data, sql

#### Acquire & Summarize

1. **Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.**

   - *Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for* 
`airconditioningtypeid`
*.*
   - Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
   - Only include properties that include a latitude and longitude value.

In [ ]:
dfo = set_dfo(get_zillow_data(), splain=True)

2019-11-06 10:29:31 starting get_zillow_data
2019-11-06 10:29:31 starting sql_df
2019-11-06 10:29:31 starting get_db_url
2019-11-06 10:29:31 ending get_db_url ; time: 0:00:00.000086


2. **Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.).**

In [ ]:
df = dfo.df

In [ ]:
frame_splain(dfo.df, title='Zillow Data', splain=True)

In [ ]:
@timeifdebug
def convert_to_dates(df, cols=[], format='%Y-%m-%d', errors='coerce', **kwargs):
    '''
    convert_to_dates(
        df, 
        cols, 
        format='%y-%m-%d', 
        errors='coerce', 
        **kwargs
    )
    RETURNS df with columns updated to dates
    '''
    convertcols = [col for col in cols if col in df.columns]
    for col in convertcols:
        df[col] = pd.to_datetime(df[col], errors=errors, format=format)
    return df

In [ ]:
df = convert_to_dates(df, cols=['transactiondate'])

In [ ]:
dfo.df.head(15)

In [ ]:
frame_splain(dfo.df, splain=True)

3: **Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.**

||num_rows_missing|pct_rows_missing|
|:---|---:|---:|
|parcelid|0|0.000000|
|airconditioningtypeid|29041|0.535486|
|architecturalstyletypeid|54232|0.999982|


In [ ]:
@timeifdebug
def get_column_values_stats(
        df, 
        get_cols=[], 
        max_uniques=10, 
        target_col='',
        limit_to_max=False,
        **kwargs
        ):
    '''
    get_column_values_stats(
        df, 
        get_cols=[], 
        max_uniques=10, 
        target_col='', 
        limit_to_max=False,
        **kwargs
        )
    RETURNS summary dataframe

    Receives dataframe as input, examines all columns defined as objects, and
    returns a summary report with column name as its index.

    Report shows the following information about the columns of the dataframe:
        cols: column name
        dtypes: data type
        num_rows_values: number of rows with non-null values 
        num_rows_missing: number of rows with missing values
        pct_rows_missing: percentage of rows with missing values
        num_uniques: number of unique values 
        unique_values: list of unique values if the unique value count is less than 
            or equal to max_uniques.
        
    If the input dataframe contains the target column, enter that name as the 
    target_col argument and it will be removed from the analysis.
    
    If limit_to_max is True (default), the resulting dataframe will only show
    columns with no more unique values than specified in max_uniques.
    
    NOTE: This function can be used to identify and then re-type catergorical
    columns. Run the function with the following parameters, change the data 
    types of know categorical values to 'category', then re-run this function, 
    gradually increasing the max_uniques value until the desired limit for 
    categoricals has been met.
    
    get_column_values_stats(
        df, 
        get_cols=df.columns[df.dtypes!='category'],
        max_uniques=5,
        limit_to_max=True
        )
    '''
    # get rowcount
    num_recs = len(df)
    
    # if no columns presented, get all columns from df
    use_cols = df.columns if len(get_cols) == 0 else get_cols
    # ensure use_cols are actually present in df
    cols = [col for col in use_cols if col in df.columns]
    
    # make df for all columns, exclude target column if passed
    df_cols = pd.DataFrame(cols, columns=['cols'])
    df_cols = df_cols[df_cols.cols != target_col]
    
    # get statistics on all columns
    df_cols['dtype'] = df_cols.cols.apply(lambda x: df[x].dtype)
    df_cols['num_rows_values'] = df_cols.cols.apply(lambda x: df[x].count())
    df_cols['num_rows_missing'] = num_recs - df_cols.num_rows_values
    df_cols['pct_rows_missing'] = df_cols.num_rows_missing / num_recs
    df_cols['num_uniques'] = df_cols.cols.apply(lambda x: df[x].nunique())
    
    # get unique valeues for columns within unique value limits
    df_cats = df_cols[df_cols.num_uniques <= max_uniques]
    df_cats['unique_values'] = df_cats.cols.apply(lambda x: df[x].unique())
    
    # merge and set index
    join_type = 'inner' if limit_to_max else 'left'
    df_cols = df_cols.join(df_cats.unique_values, how=join_type)
    return df_cols.set_index('cols')

**Find string categoricals**

In [ ]:
df_strings = get_column_values_stats(
    df, 
    get_cols=df.columns[df.dtypes=='object'],
    max_uniques=20000,
    limit_to_max=True
)
df_strings

In [ ]:
df_strings.index

In [ ]:
string_categoricals = [
    'airconditioningdesc', 
    'architecturalstyledesc', 
    'buildingclassdesc', 
    'county', 
    'state', 
    'heatingorsystemdesc', 
    'propertycountylandusecode', 
    'propertylandusedesc', 
    'propertyzoningdesc', 
    'storydesc', 
    'taxdelinquencyflag', 
    'typeconstructiondesc'
]

In [ ]:
df = prep.retype_cols(df, cols=string_categoricals, to_dtype='category')
df.info()

**Find Numeric Categoricals**

In [ ]:
df_cat_check = get_column_values_stats(
    df, 
    get_cols=df.columns[df.dtypes!='category'],
    max_uniques=750000,
    limit_to_max=True
)
df_cat_check

In [ ]:
df_cat_check.index

In [ ]:
numeric_categoricals = [
    'airconditioningtypeid', 
    'architecturalstyletypeid', 
    'buildingclasstypeid', 
    'buildingqualitytypeid',
    'decktypeid', 
    'fips', 
    'hashottuborspa',
    'heatingorsystemtypeid',
    'pooltypeid10', 
    'pooltypeid2', 
    'pooltypeid7', 
    'propertylandusetypeid',
    'regionidcounty', 
    'regionidcity',
    'regionidneighborhood',
    'regionidzip',
    'rawcensustractandblock',
    'censustractandblock',
    'storytypeid', 
    'threequarterbathnbr', 
    'numberofstories', 
    'fireplaceflag', 
    'assessmentyear', 
    'typeconstructiontypeid', 
    'transactions'
]

In [ ]:
df = prep.retype_cols(df, cols=numeric_categoricals, to_dtype='category')
df.info()

4. **Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.**

num_cols_missing|pct_cols_missing|num_rows
:---|---:|---:
23|38.333|108
24|40.000|123
25|41.667|5280

In [ ]:
def nulls_by_row(df):
    total_cols = df.shape[1]
    num_cols_missing = df.isnull().sum(axis=1)
    pct_cols_missing = df.isnull().sum(axis=1)/ total_cols
    rows_missing = (
        pd.DataFrame(
        {
            'num_cols_missing': num_cols_missing, 
            'pct_cols_missing': pct_cols_missing
        }
        )
        .reset_index()
        .groupby(['num_cols_missing','pct_cols_missing'])
        .count()
        .rename(index=str, columns={'index': 'num_rows'})
        .reset_index()
    )
    return rows_missing 

In [ ]:
nulls_by_row(df)

#### Prepare

1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

2. Create a function that will drop rows or columns based on the percent of values that are missing: `handle_missing_values(df, prop_required_column, prop_required_row)`.

   - The input:
       - A dataframe
       - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
       - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
   - The output:
       - The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
   - hint:
       - Look up the dropna documentation.
       - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
       - Make use of inplace, i.e. inplace=True/False.

In [ ]:
def handle_missing_values(
    df, 
    prop_required_column, 
    prop_required_row
    ):
    '''
    handle_missing_values(
    df, 
    prop_required_column=.9, 
    prop_required_row=.9
    )
    '''
    

3. Decide how to handle the remaining missing values:
   - Fill with constant value.
   - Impute with mean, median, mode.
   - Drop row/column

In [ ]:
df_sqft = df[['calculatedfinishedsquarefeet',
'finishedfloor1squarefeet',
'finishedsquarefeet6',
'finishedsquarefeet12',
'finishedsquarefeet13',
'finishedsquarefeet15',
'finishedsquarefeet50'
]]

#### `wrangle_zillow.py`

Functions of the work above needed to acquire and prepare a new sample of data.

In [ ]:
df_sqft[df_sqft.calculatedfinishedsquarefeet.isna() & (df_sqft.finishedsquarefeet50.isna()==False)]

In [ ]:
df_pool = df[['poolcnt','poolsizesum','pooltypeid2','pooltypeid7','pooltypeid10']]
df_pool[(df_pool.poolcnt.isna()==False) | (df_pool.pooltypeid10.isna()==False)]